In [498]:
import pandas as pd, numpy as np, json, os
import matplotlib.pyplot as plt
%matplotlib inline

In [499]:
plt.style.use('seaborn-whitegrid')

In [500]:
#!pip install d3IpyPlus
from d3IpyPlus import ScatterPlot, LinePlot, BarPlot, StackedArea

In [501]:
#!pip install pyecharts

In [502]:
m=json.loads(open('ignore/member_timelines.json','r').read())
members=json.loads(open('ignore/member_simple.json','r').read())

In [503]:
d=pd.DataFrame(members['deaths'],index=['v']).T

In [504]:
d['v']=' †'

In [505]:
s=pd.DataFrame(members['s2'],index=['s']).T
s=s.join(d)
s['v']=s['v'].fillna('')
s['s']=s['s']+s['v']
s=s[['s']]
s['w']=s['s'].str.split(' ').str[0].str[0]+s['s'].str.split(' ').str[-1].str[0]

In [506]:
szd_color='#E91C62'

In [507]:
pcolors=json.loads(open('json/pcolors.json','r').read())

In [508]:
g=pd.DataFrame(members['megye'],index=['Megye ']).T
g.head()

,Megye
Scrieciu Gheorghe | 1932-09-11,🇷🇴 Dolj
Bejan Costache | 1946-08-13,🇷🇴 Iași
Armasescu Dumitru | 1947-02-25,🇷🇴 Hunyad
Negoescu Nicolae-Ioan | 1933-06-24,🇷🇴 Temes
Lupau Gheorghita | 1945-12-08,🇷🇴 Bihar


In [509]:
data={}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        if i['Csoportok'] not in data:data[i['Csoportok']]={'start':pd.to_datetime('2030'),'end':pd.to_datetime('1980')}
        start=pd.to_datetime(i['start'])
        end=pd.to_datetime(i['end'])
        data[i['Csoportok']]['start']=min(data[i['Csoportok']]['start'],start)
        data[i['Csoportok']]['end']=max(data[i['Csoportok']]['end'],end)

0.0 %
3.3028371371007696 %
6.605674274201539 %
9.908511411302309 %
13.211348548403079 %
16.51418568550385 %
19.817022822604617 %
23.119859959705387 %
26.422697096806157 %
29.725534233906924 %
33.0283713710077 %
36.33120850810847 %
39.634045645209234 %
42.93688278231001 %
46.239719919410774 %
49.54255705651154 %
52.845394193612314 %
56.14823133071308 %
59.45106846781385 %
62.753905604914614 %
66.0567427420154 %
69.35957987911617 %
72.66241701621693 %
75.9652541533177 %
79.26809129041847 %
82.57092842751923 %
85.87376556462002 %
89.17660270172078 %
92.47943983882155 %
95.78227697592231 %
99.08511411302308 %


In [510]:
part_timelines=pd.DataFrame(data).T
part_timelines.index.name='id'

In [511]:
part_timelines['start']=part_timelines['start'].astype(str).str[:10]
part_timelines['end']=part_timelines['end'].astype(str).str[:10]

In [512]:
p=part_timelines.reset_index()[['id']]
p['Color']=p['id'].str.split(' ').str[1].str.strip()
p['Part']=p['id'].str.split('-').str[0].str.strip()

In [513]:
part_timelines=part_timelines.join(p.set_index('id').drop_duplicates())

In [514]:
logos={'🕊️':'ALDE', '🌹':'PSD', '📐':'PNL', '🔶':'PD', '⚛️':'PP-DD', '👤':'Független', '⚪️':'FSN',
       '✳️':'PNTCD', '🔱':'PRM', '🌷':'RMDSZ', '🍀':'ECO', '🍏':'PMP',
       '🔷':'USR', '🦅':'UNPR', '🔘':'Más'}

In [515]:
part_timelines['logo']=part_timelines['Part'].str.split(' ').str[0]

In [516]:
part_timelines['group']=[logos[i] for i in part_timelines['logo'].values]
part_timelines['label']=[i+' '+logos[i] for i in part_timelines['logo'].values]

In [517]:
part_timelines=part_timelines.join(pd.DataFrame(pcolors,index=['cvalue']).T,on='group').fillna('grey')

In [518]:
part_timelines_list=list(part_timelines.reset_index().T.to_dict().values())

In [519]:
open('json/part_timelines.json','w').write(json.dumps(part_timelines_list))

9874

Resolve for non-activity


In [520]:
from calendar import monthrange

In [521]:
data={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        part=i['Csoportok']
        if part not in data:data[part]={}
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:7]
            if st not in data[part]:data[part][st]=0
            data[part][st]+=1/monthrange(t.year,t.month)[1]

0.0 %
3.3028371371007696 %
6.605674274201539 %
9.908511411302309 %
13.211348548403079 %
16.51418568550385 %
19.817022822604617 %
23.119859959705387 %
26.422697096806157 %
29.725534233906924 %
33.0283713710077 %
36.33120850810847 %
39.634045645209234 %
42.93688278231001 %
46.239719919410774 %
49.54255705651154 %
52.845394193612314 %
56.14823133071308 %
59.45106846781385 %
62.753905604914614 %
66.0567427420154 %
69.35957987911617 %
72.66241701621693 %
75.9652541533177 %
79.26809129041847 %
82.57092842751923 %
85.87376556462002 %
89.17660270172078 %
92.47943983882155 %
95.78227697592231 %
99.08511411302308 %


In [538]:
df=pd.DataFrame(data)

In [539]:
df=df.stack().reset_index()
df.columns=['x','id','y']

In [540]:
df=df.groupby(['id','x']).mean().rolling(7).mean().reset_index().dropna()

In [541]:
df2=df.join(part_timelines,on='id')

In [542]:
df2=df2[df2['y']>2]
values=list(df2.T.to_dict().values())
open('json/part_timelines2.json','w').write(json.dumps(values))

966954